In [28]:
import pymysql





 
# MySQL Connection 연결
conn = pymysql.connect(host='localhost', user='root', password='123456789',
                       db='lafamila', charset='utf8')
 
# Connection 으로부터 일반 Cursor 생성
curs = conn.cursor()

# Connection 으로부터 Dictoionary Cursor 생성
#curs = conn.cursor(pymysql.cursors.DictCursor)
 
# SQL문 실행
sql = "select * from test"
curs.execute(sql)


#insert 나 update와 같은 query는 commit 과정이 필요하다
#sql = "insert into test values('asdf', 'kk', '@mai.com', 4321)"
#curs.execute(sql)
#conn.commit()


# sql에 변수 넣기
# sql query를 구성할 때 만약 String Formatting 을 사용하게되면 injection 등의 문제를 일으킬 수 있다.
# 따라서 데이터가 들어갈 자리를 %s로 대체해두고 cursor를 실행할 때 tuple 형식으로 첨부한다.
# (이때 %s는 일반적인 formatting 이 아님)
#sql = "select * from test where PW=%s"
#curs.execute(sql, ('adf'))


# 데이타 Fetch
rows = curs.fetchall()

# 하나 가져올때는 fetchone()
# 여러개 가져올때는 fetchmany(n) 

# 가져온 data는 Cursor의 종류에 따라 tuple 또는 dict 가 된다.
for row in rows:    
    print(row, type(row))     # 전체 rows 출력

# Connection 닫기
conn.close()

In [34]:
import pymysql
 
conn = pymysql.connect(host='localhost', user='root', password='123456789',
                       db='lafamila', charset='utf8')
 
curs = conn.cursor(pymysql.cursors.DictCursor) 
sql = "select * from test"
curs.execute(sql)
rows = curs.fetchall()

print(type(rows))
for row in rows:    
    
    print(row["ID"])
    #print(row, type(row))

    
    
conn.close()

<class 'list'>
asdf
lafa


In [36]:
import pymysql
 
conn = pymysql.connect(host='localhost', user='root', password='123456789',
                       db='shoppingmall', charset='utf8')
 
curs = conn.cursor(pymysql.cursors.DictCursor) 
sql = "insert into category_update values(NOW(), %s, %s, %s)"
curs.execute(sql, ("name", 0, 0))
conn.commit()
#rows = curs.fetchall()

#print(type(rows))
#for row in rows:    
    
#    print(row["ID"])
    #print(row, type(row))

    
    
conn.close()

# Shopping Mall

In [61]:
import requests
from bs4 import BeautifulSoup
import urllib
import pymysql
IMAGE_PATH = "C:\\Users\\lafamila\\Downloads\\"

conn = pymysql.connect(host='localhost', user='root', password='123456789',
                       db='shoppingmall', charset='utf8')


curs = conn.cursor() 
sql = "select p_number from product_update"
# sql query를 구성할 때 만약 String Formatting 을 사용하게되면 injection 등의 문제를 일으킬 수 있다.
# 따라서 데이터가 들어갈 자리를 %s로 대체해두고 cursor를 실행할 때 tuple 형식으로 첨부한다.
curs.execute(sql)
rows = curs.fetchall()




#top에서 에러가 나서 top만 실행해보았음
url_d = {   
    'top':'http://www.secretlabel.co.kr/shop/shopbrand.html?xcode=047',
}
for url in url_d.values():
    p_nums = set()
    category_update = 0
    response = requests.get(url)
    soup = BeautifulSoup(response.content,'html.parser')
    #print(soup.title)
    p_count = 1
    page_url = url + soup.select_one('.now > a').get('href')[0:-1]
    #print(page_url)
    product_count = 0
    
    while True:
        page_url_v = page_url + str(p_count)
        p_count += 1
        res = requests.get(page_url_v)
        soup = BeautifulSoup(res.content,'html.parser')
        soup_p = soup.select_one('#SMS_prd_container')
        product_lists = soup_p.select('.item_list')
        product_count += len(product_lists)
        
        
        if len(soup_p.select('.item_list')) == 0:
            break
        for y in product_lists:
            li = y.find_all("li")

            #상품이미지 다운로드 및 경로
            image_src = y.select_one('.MS_prod_img_s').get('src').split('?')[0]
            start = image_src.rindex("/")+1
            #urllib.request.urlretrieve( image_src , IMAGE_PATH+image_src[start:] ) 
            
            #이름
            name = y.select_one('.pname').text.strip('\n').split()[0]

            
            #금액
            price = y.select_one('.price').text.strip()
            try:
                price = int(price[:-1].replace(",",""))
            except:
                print(price)
                break

              
            
            #url
            product_url = url + y.select_one('a').get('href')
            
            #상품후기수
            #상품번호를 받아와서 하면 된다.
            p_num = y.select_one('a').get('href')[31:37]
            
            #cre = 'http://widgets3.cre.ma/secretlabel.co.kr/products/reviews?product_code=' + p_num
            #cre_r = requests.get(cre)
            #cre_soup = BeautifulSoup(cre_r.content,'html.parser')

            #총리뷰를 불러와라
            #review_count = cre_soup.select_one('.reviews-count').text
            
            #상품키
            p_num = url.split(".")[1] + p_num
            
            p_nums.add((p_num,))
            
            if (p_num,) in rows:
                print("update")
            else:            
                """
                ###########################################################
                ##                          DB                           ##
                ###########################################################
                # p_num
                # name
                # price
                # product_url
                # IMAGE_PATH+image_src[start:]
                # 'top'
                # review_count
                # '시크릿라벨'

                curs = conn.cursor(pymysql.cursors.DictCursor) 

                #INSERT
                sql = "insert into product_update values(NOW(), %s, %s, %s, %s, %s, %s, %s, %s)"
                curs.execute(sql, (p_num, name, price, product_url, IMAGE_PATH+image_src[start:], 'top', review_count, "시크릿라벨"))
                conn.commit()
                """
                category_update += 1
            

        #print('카테고리상품수 :' + str(product_count) + '개')
    
    
    #기존에 있던 상품key 들의 집합에서 새로 크롤링한 상품key 들의 차집합을 구함
    #--> 기존에는 있었으나 새로 크롤링한 데이터에는 없는 개수 (품절개수)
    print(len(set(list(rows)).difference(p_nums)))


    ###########################################################
    ##                          DB                           ##
    ###########################################################
    # 'top'
    # category_update
    # len(set(list(rows)).difference(p_nums))
    curs = conn.cursor(pymysql.cursors.DictCursor) 

    #INSERT
    sql = "insert into category_update values(NOW(), %s, %s, %s)"
    curs.execute(sql, ('top', category_update, len(set(list(rows)).difference(p_nums))))
    conn.commit()

    
conn.close()

update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
update
0


In [57]:
print(set(list(rows)))
print(p_nums)

{('secretlabel516072',), ('secretlabel515815',), ('secretlabel512031',), ('secretlabel515922',), ('secretlabel511826',), ('secretlabel516071',), ('secretlabel515889',), ('secretlabel516197',), ('secretlabel513144',), ('secretlabel515523',), ('secretlabel511853',), ('secretlabel513301',), ('secretlabel515872',), ('secretlabel516096',), ('secretlabel512346',), ('secretlabel513176',), ('secretlabel511950',), ('secretlabel515856',), ('secretlabel515865',), ('secretlabel516128',), ('secretlabel516037',), ('secretlabel515940',), ('secretlabel515986',), ('secretlabel516104',), ('secretlabel513058',), ('secretlabel515971',), ('secretlabel516133',), ('secretlabel516026',), ('secretlabel515973',), ('secretlabel516053',), ('secretlabel515955',), ('secretlabel512503',), ('secretlabel516031',), ('secretlabel515884',), ('secretlabel516035',), ('secretlabel516200',), ('secretlabel511237',), ('secretlabel511406',), ('secretlabel512809',), ('secretlabel515957',), ('secretlabel516117',), ('secretlabel51